In [1]:
import pandas as pd
def build_empirical(df_counts):
    '''
    Given a value counts dataframe, return a dataframe indexed by demand and containing columns count, pmf value, and cmf value
    '''
    demand_distribution = pd.DataFrame(df_counts)
    demand_distribution.sort_index(inplace=True)
    demand_distribution['pmf'] = demand_distribution['count']/demand_distribution['count'].sum()
    demand_distribution['cmf'] = demand_distribution['pmf'].cumsum()
    return demand_distribution
def update_empirical(empirical, demand):
    '''
    Update empirical demand distribution with new uncensored demand
    '''
    # Safety check: ensure empirical is not empty
    if empirical.empty:
        # Create new empirical from this single observation
        empirical = build_empirical(pd.Series([demand]).value_counts())
        return empirical
    if demand in empirical.index:
        empirical.loc[demand, 'count'] +=1
    else:
        empirical.loc[demand, 'count']=1
        empirical.sort_index(inplace=True)
    
    # Recalculate PMF and CMF
    total_count = empirical['count'].sum()
    if total_count > 0:
        empirical['pmf'] = empirical['count'] / total_count
        empirical['cmf'] = empirical['pmf'].cumsum()
    return empirical

def apply_inventory(demand_df, methods, critical_ratio = 0.9):
    df = demand_df.copy()
    df[[method + "_Q" for method in methods]] = None #initialize new inventory columns
    df = demand_df.set_index(['Magazine', 'EHASTRA_EH_NUMMER', 'Period'])
    df.sort_index(inplace=True)
    
    df.dropna(inplace=True)
    
    for (magazine, pos), group in df.groupby(['Magazine', 'EHASTRA_EH_NUMMER']):
        print(f'Running KPIs for magazine {magazine}, pos {pos}')
        df_train = group[group['Heftjahr']<2024]
        if df_train.empty: continue

        for method in methods:
            # initialize empirical distribution
            empirical = build_empirical(df_train[method+"_Demand"].value_counts())

            # suggest inventory for 2024
            df_test = group[group['Heftjahr']==2024]
            if df_test.empty: continue
            for row in df_test.itertuples():
                magazine, pos, period = row.Index
                df.loc[(magazine, pos, period), method + '_Q'] = (empirical['cmf']>=0.9).idxmax()
                empirical = update_empirical(empirical, getattr(row, method+"_Demand"))

    return df

In [2]:
df_demand = pd.read_csv('Uncensoring_results.csv')
methods = ['N1', 'N2', 'N3', 'EM', 'PD', 'Nahmias', 'Conrad', 'Baseline']
df_inventory = apply_inventory(df_demand, methods)

df_inventory.to_csv('Inventory_results.csv')

Running KPIs for magazine A, pos EHA0017186
Running KPIs for magazine A, pos EHA0021360
Running KPIs for magazine A, pos EHB0019844
Running KPIs for magazine A, pos EHB0023505
Running KPIs for magazine A, pos EHC0004006
Running KPIs for magazine A, pos EHC0005070
Running KPIs for magazine A, pos EHE0003391
Running KPIs for magazine A, pos EHH0010681
Running KPIs for magazine B, pos EHA0020265
Running KPIs for magazine B, pos EHB0023505
Running KPIs for magazine B, pos EHD0015991
Running KPIs for magazine B, pos EHG0003391
Running KPIs for magazine C, pos EHB0023505
Running KPIs for magazine D, pos EHA0021360
Running KPIs for magazine D, pos EHB0019844
Running KPIs for magazine D, pos EHH0010681
Running KPIs for magazine E, pos EHB0019622
Running KPIs for magazine E, pos EHC0005070
Running KPIs for magazine E, pos EHE0003391
Running KPIs for magazine F, pos EHG0006538
Running KPIs for magazine G, pos EHB0023505
Running KPIs for magazine G, pos EHG0006538
Running KPIs for magazine H, pos